# Relevant Bits of Code for Napari 

## Data Preparation
### Unstacking 

This code snippet helps unstack a stacked tif file with txy format. 

In [ ]:
import imageio
import os

def extract_tiff_frames_with_imageio(tiff_path, output_folder):
    reader = imageio.get_reader(tiff_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, im in enumerate(reader):
        if im.ndim == 2:  
            imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}.tif", im)
        elif im.ndim == 3:  
            for j, slice in enumerate(im):
                imageio.imwrite(f"{output_folder}/frame_{str(i+1).zfill(4)}_slice_{str(j+1).zfill(4)}.tif", slice)
        else:
            print(f"Frame {i+1} has unsupported dimensions: {im.ndim}")

data = r"D:\documents\digibio\data\image_intensities_(well)_unstacked"
file = r"D:\documents\digibio\data\amdtrk\x1_imageintensity.tif"

extract_tiff_frames_with_imageio(file, data)